In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
reload(fd)
reload(sd)
reload(di)

print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

# Test with default hyperparameters

In [ ]:
from imblearn.ensemble import EasyEnsembleClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = EasyEnsembleClassifier()
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)

# result 28/01/2025
# Duration 43,0 s
#train f1 score: 0.0152 mcc score: 0.0744 roc auc score: 0.8698
#test f1 score: 0.0152 mcc score: 0.0741 roc auc score: 0.8660

# Final validation

In [ ]:
files = fd.getAllFiles()

range = []
results = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)

    preds = modelClf.predict(dfTrx[predictors])

    result= fd.print_scores(dfTrx['Class'], preds,'f1', True)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    results.append(result)

fd.plt_train_test(range, results)

#test result (22/01/2025)
# d1 f1= 0.1293
# d2 f1= 0.0169
# d3 f1= 0.0270
# d7 f1= 0.5730